# Recommendation System

In [1]:
import pandas as pd

df_credits = pd.read_csv('../datasets/tmdb_5000_credits.csv', low_memory=False)
df_movies = pd.read_csv('../datasets/tmdb_5000_movies.csv', low_memory=False)

#Объединяем df
df = pd.merge(df_credits, df_movies, left_on='movie_id', right_on='id', how='outer')

#Оставляем только фильмы, вышедшие в релиз.
df = df[df.status == 'Released']
print(f"Количество фильмов, оставшихся после фильтрации: {df.shape[0]}")

Количество фильмов, оставшихся после фильтрации: 4795


In [2]:
# Функция расчёта рейтинга IMDB
def weighted_rating(row):
    v = row['vote_count']
    r = row['vote_average']
    wr = v / (v + m) * r + m / (v + m) * c
    return wr

In [3]:
# Cредняя оценка всех фильмов
c = (df['vote_average']).mean()

# Количество голосов для включения в финальную таблицу
m = df['vote_count'].quantile (.95)

# Запускаем функцию расчёта рейтинга IMDB для каждого фильма 
df['simple_score'] = df.apply(weighted_rating, axis=1)

# Сортируем датафрейм по рейтингу
df = df.sort_values(by = ['simple_score'], ascending=False)

# Топ-5 фильмов по рейтингу
top_5 = list(df[:5]['title_x'])

print('Топ-5 фильмов:')
print(*top_5, sep = ', ')

Топ-5 фильмов:
The Shawshank Redemption, The Dark Knight, Fight Club, Inception, Pulp Fiction


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Замена NaN на '' в столбце с описанием фильма
df['overview'] = df['overview'].fillna('')

# Исключение неинформотивных слов, редковстречающихся слов, небуквенных символов, привидение текста к нижнему регистру.
# Векторизируем столбец с описанием фильма
vectorizer = TfidfVectorizer(stop_words='english')
tf_idf = vectorizer.fit_transform(df['overview'])

print('Размер матрицы Tf-Idf:', tf_idf.shape)

Размер матрицы Tf-Idf: (4795, 20970)


In [5]:
from sklearn.metrics.pairwise import linear_kernel

# Оценка косинусного сходства
cosine_sim = linear_kernel(tf_idf)

print('Размер матрицы cosine_sim:', cosine_sim.shape)

Размер матрицы cosine_sim: (4795, 4795)


In [6]:
#Функция поиска схожих фильмов по описанию
def get_recommendations(movies_dataset, title, cosine_sim, top_k=10):
    
    #Получаем индекс выбранного фильма
    movie_index = movies_dataset[movies_dataset['title_x']==title].index[0]
    
    #Косинусное расстояние до выбранного фильма
    distances = cosine_sim[movie_index]
    
    #Список фильмов отсортированный по сходству фильмов с выбранным фильмом
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:top_k+1]
    
    #Выводим названия фильмов наиболее похожих по описанию на выбранный фильм
    for i in movies_list:
        print(movies_dataset.iloc[i[0]].title_x)

In [7]:
#Датасет с названием и описанием фильмов
dataset = df[['title_x', 'overview']]
#Сброс индекс, задаём список от 0
dataset.reset_index(drop= True , inplace= True )

#Заупскаем функцию поиска схожих фильмов по описанию для фильма Saving Private Ryan
similar_films = get_recommendations(movies_dataset=dataset, title='Saving Private Ryan', cosine_sim=cosine_sim, top_k=5)

The Great Raid
The Monuments Men
The Expendables 2
Abandoned
The Train


In [8]:
# conda install -c conda-forge scikit-surprise - для установки библиотеки

In [9]:
import pandas as pd
import numpy as np

from surprise import Dataset, NormalPredictor, Reader, SVD
from surprise.model_selection import cross_validate

df_ratings = pd.read_csv('../datasets/ratings.csv', low_memory=False)


# Шкала рейтинга (от 0 до 5)
reader = Reader(rating_scale=(0, 5))

# Загружаем датасет из dataframe pandas, указываем три столбца: id пользователя, id предмета оценки и оценка.
data = Dataset.load_from_df(df_ratings[["userId", "movieId", "rating"]], reader)

# Используем SVD алгоритм (сингулярное разложении матриц).
algo = SVD()

# Запустим пятикратную перекрестную проверку и выведем результат.
cross_valid = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# RMSE - Root Mean Squared Error Среднеквадратичная ошибка
# MAE - Mean Absolute Error Средняя абсолютная ошибка
print("\nСреднее значение 'RMSE':", round(np.mean(cross_valid['test_rmse']), 4))
print("Среднее значение 'MAE':", round(np.mean(cross_valid['test_mae']), 4))

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8959  0.8963  0.9013  0.8937  0.8995  0.8973  0.0027  
MAE (testset)     0.6893  0.6895  0.6944  0.6901  0.6913  0.6909  0.0019  
Fit time          0.89    0.90    0.91    0.91    0.92    0.91    0.01    
Test time         0.18    0.11    0.10    0.18    0.11    0.14    0.03    

Среднее значение 'RMSE': 0.8973
Среднее значение 'MAE': 0.6909
